# <center>Projet Neo4j - PapersWithCode</center>

**<center>Lucien DEROUET - Antoine PURIER</center>**

## Introduction

&emsp;&emsp;Dans le cadre de notre projet neo4j, nous construisons une base de données graphe à partir des metadonnées de publications hébergées sur le site https://paperswithcode.com. Des informations supplémentaires sur ces publications sont également récupérées sur d'autres sites via des API et du scraping.

Le but est ensuite de requêter cette base pour répondre à des questions sur le dataset, telles que:
- REMPLACER PAR REQUÊTE 1
- REMPLACER PAR REQUÊTE 2
- REMPLACER PAR REQUÊTE 3
- ...

## Les Données

&emsp;&emsp;Pour la récupération des données et la construction de notre dataset, nous partons du site https://paperswithcode.com. Son API publique nous permet facilement de récupérer les métadonnées des plus de 900.000 publications qu'il héberge.

&emsp;&emsp;Les metadonnées pour chaque papier sont stockées dans un dictionnaire qui est exporté au format json. Nous construirons notre BDD graphe à partir de ce fichier json d'entrée.

&emsp;&emsp;Ci-dessous un flowchart illustrant les différents briques mises en place pour la construction du dataset.

![title](flowchart.png)

Le code ayant servi à récupérer et à construire le dataset se trouve en annexe dans le dossier de rendu.

Dans la prochaine section, nous nous connectons à Neo4j et nous créons la base de donnée.

## Connexion à Neo4j

Nous commençons par vérifier la version de neo4j installée sur la machine.

In [1]:
from neo4j import __version__ as neo4j_version
print(neo4j_version)

4.4.1


Nous créons ensuite une classe pour la connexion à neo4j.

In [3]:
from neo4j import GraphDatabase

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

Puis nous instantions une connexion.

In [4]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="antoine", pwd="easypassword")

## Création de la Base

Nous créons maintenant la base de données graphe `paperswithcode`.

In [84]:
conn.query("CREATE OR REPLACE DATABASE paperswithcode")

[]

Puis nous créons les noeuds et les relations du graphe à partir du fichier json.

Nous créons 6 noeuds:
- `Paper`
- `Author`
- `Publisher`
- `Organisation`
- `Conference`
- `Keyword`

Et nous relions ces noeuds à l'aide de 6 relations:
- Un auteur a écrit un papier : `WROTE`
- Un papier a été présenté à une conférence : `PRESENTED_AT`
- Un éditeur a publié un papier : `PUBLISHED`
- Un auteur est affilié à une organisation (université, entreprise, ...) : `AFFILIATED_TO`
- Un papier parle d'un sujet (évoque un mot clé) : `TALKS_ABOUT`
- Un papier fait référence à un autre papier : `REFERS_TO`

In [86]:
# Import du module `time` pour chronométrer nos requêtes
import time

# Fichier à charger (à placer dans le dossier `Import`)
file = "dataset_18K_v7.json"

### Création des noeuds `Paper`

In [87]:
query_string = '''
CREATE CONSTRAINT ON (p:Paper) ASSERT p.doi IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [88]:
query_string = '''
CREATE CONSTRAINT ON (p:Paper) ASSERT exists(p.doi)
'''
conn.query(query_string, db='paperswithcode')

[]

In [89]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"MERGE (p:Paper {{title: paper.title}})
SET
    p.doi = paper.doi,
    p.language = paper.language,
    p.arxiv_category = paper.arxiv_category,
    p.publication_date = paper.date,
    p.references = paper.references",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

26.54481291770935


### Création des noeuds `Author`

In [90]:
query_string = '''
CREATE CONSTRAINT ON (a:Author) ASSERT a.name IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [91]:
query_string = '''
CREATE CONSTRAINT ON (a:Author) ASSERT exists(a.name)
'''
conn.query(query_string, db='paperswithcode')

[]

In [92]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"WITH paper.authors as authors
UNWIND
    CASE
        WHEN authors = [] THEN [null]
        ELSE authors
    END AS aut
WITH aut
WHERE aut IS NOT NULL
MERGE (a:Author {{name: aut.name}})",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

3.779695987701416


### Création des noeuds `Organisation`

In [93]:
query_string = '''
CREATE CONSTRAINT ON (o:Organisation) ASSERT o.name IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [94]:
query_string = '''
CREATE CONSTRAINT ON (o:Organisation) ASSERT exists(o.name)
'''
conn.query(query_string, db='paperswithcode')

[]

In [95]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"WITH paper.authors as authors
UNWIND authors as aut
UNWIND
    CASE
        WHEN aut.organisation = [] THEN [null]
        ELSE aut.organisation
    END AS org
WITH org
WHERE org IS NOT NULL
MERGE (o:Organisation {{name: org.name}})",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

4.339322090148926


### Création des noeuds `Keyword`

In [96]:
query_string = '''
CREATE CONSTRAINT ON (k:Keyword) ASSERT k.name IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [97]:
query_string = '''
CREATE CONSTRAINT ON (k:Keyword) ASSERT exists(k.name)
'''
conn.query(query_string, db='paperswithcode')

[]

In [98]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"WITH paper.key_words as keywords
UNWIND
    CASE
        WHEN keywords = [] THEN [null]
        ELSE keywords
    END AS key
WITH key 
WHERE key IS NOT NULL
MERGE (k:Keyword {{name: key}})",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

3.759504795074463


### Création des noeuds `Publisher`

In [99]:
query_string = '''
CREATE CONSTRAINT ON (pub:Publisher) ASSERT pub.name IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [100]:
query_string = '''
CREATE CONSTRAINT ON (pub:Publisher) ASSERT exists(pub.name)
'''
conn.query(query_string, db='paperswithcode')

[]

In [101]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"MERGE (pub:Publisher {{name: paper.publisher}})",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

3.189958095550537


### Création des noeuds `Conference`

In [102]:
query_string = '''
CREATE CONSTRAINT ON (c:Conference) ASSERT c.name IS UNIQUE
'''
conn.query(query_string, db='paperswithcode')

[]

In [103]:
query_string = '''
CREATE CONSTRAINT ON (c:Conference) ASSERT exists(c.name)
'''
conn.query(query_string, db='paperswithcode')

[]

In [104]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"WITH paper.conference as conference
MERGE (c:Conference {{name: conference.name}})
SET
    c.date = conference.date,
    c.location = conference.location",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

2.3070120811462402


### Création des relations entre les noeuds `Paper` et `Author`

In [105]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"UNWIND paper.authors as aut
MATCH (p:Paper {{doi: paper.doi}}), (a:Author {{name: aut.name}})
MERGE (a)-[:WROTE]->(p)",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

4.892545938491821


### Création des relations entre les noeuds `Paper` et `Keyword`

In [106]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"UNWIND
    CASE
        WHEN paper.key_words = [] THEN [null]
        ELSE paper.key_words
    END AS key
MATCH (p:Paper {{doi: paper.doi}}), (k:Keyword {{name: key}})
MERGE (p)-[:TALKS_ABOUT]->(k)",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

4.991285085678101


### Création des relations entre les noeuds `Paper` et `Publisher`

In [107]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"MATCH (p:Paper {{doi: paper.doi}}), (pub:Publisher {{name: paper.publisher}})
MERGE (pub)-[:PUBLISHED]->(p)",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

4.750810146331787


### Création des relations entre `Paper` et `Conference`

In [109]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"WITH paper, paper.conference as conference
MATCH (p:Paper {{doi: paper.doi}}), (c:Conference {{name: conference.name}})
MERGE (p)-[:PRESENTED_AT]->(c)",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

6.086405992507935


### Création des relations entre `Author` et `Organisation`

In [111]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"UNWIND paper.authors as aut
UNWIND
    CASE
        WHEN aut.organisation = [] THEN [null]
        ELSE aut.organisation
    END AS org
MATCH (a:Author {{name: aut.name}}), (o:Organisation {{name: org.name}})
MERGE (a)-[:AFFILIATED_TO]->(o)",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

4.136967897415161


### Création des relations entre `Paper` et `References`

In [112]:
start = time.time()
query_string = f'''
CALL apoc.periodic.iterate(
"CALL apoc.load.json('file:{file}') YIELD value as paper",
"MATCH (p:Paper {{doi: paper.doi}}), (r:Paper)
WHERE r.doi IN p.references
CREATE (p)-[:REFERS_TO]->(r)",
{{batchSize:100, parallel:true}})
'''
conn.query(query_string, db='paperswithcode')
end = time.time()
elapsed = end - start
print(elapsed)

4.433500051498413


## Requête de la Base

Maintenant que la base est créée et notre graphe modélisé, nous la requêtons.

In [12]:
import pandas as pd

### Requête 1: Combien y-a-t-il de noeuds distincts pour chaque label?

In [113]:
labels = ["Paper", "Author", "Publisher", "Organisation", "Conference", "Keyword"]

d = {"Nombre de Noeuds": pd.NA}
df_res = pd.DataFrame(data=d, index=labels)

for label in labels:
    query_string = f'''
    MATCH (p:{label})
    RETURN count(*)
    '''
    res = [dict(_) for _ in conn.query(query_string, db='paperswithcode')]
    df_res.loc[label, "Nombre de Noeuds"] = res[0]["count(*)"]
    
display(df_res)   

,Nombre de Noeuds
Paper,18300
Author,22460
Publisher,331
Organisation,4588
Conference,34
Keyword,265


### Requête 2: Quels sont les papiers les plus référencés?

In [118]:
query_string = '''
MATCH (p:Paper)-[rel:REFERS_TO]->(r:Paper)
RETURN r.title, count(rel) as count
ORDER BY count desc
LIMIT 5
'''

df_res = pd.DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res = df_res.rename(columns={"r.title":"Titre du Papier", "count":"Nombre de Référencements"})
df_res

,Titre du Papier,Nombre de Référencements
0,"Webs of (p,q) 5-branes, five dimensional field...",16
1,A Fundamental Relation between Supermassive Bl...,13
2,SExtractor: Software for source extraction,13
3,emcee: The MCMC Hammer,12
4,"The relationship between infrared, optical, an...",11


### Requête 3: Quelles sont les organisations les plus représentées dans le dataset?

In [115]:
query_string = '''
MATCH (o:Organisation)
RETURN o.name, count(o) as count
ORDER BY count desc
LIMIT 3
'''

df_res = pd.DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res = df_res.rename(columns={"o.name":"Organisation", "count":"Auteurs Affiliés"})
df_res

,Organisation,Auteurs Affiliés
0,*Center for Bits and Atoms and,1
1,"*Department of Biological Sciences, Stanford U...",1
2,"**Departments of Human Genetics, and Psychiatr...",1


### Requête 4: Quels sont les mots clé les plus cités?

In [116]:
query_string = '''
MATCH (p:Paper)-[rel:TALKS_ABOUT]->(k:Keyword)
RETURN k.name, count(rel) as count
ORDER BY count desc
LIMIT 5
'''

df_res = pd.DataFrame([dict(_) for _ in conn.query(query_string, db='paperswithcode')])
df_res = df_res.rename(columns={"k.name":"Mot-clé", "count":"Nombre de Citations"})
df_res

,Mot-clé,Nombre de Citations
0,Astronomy and Astrophysics,706
1,Space and Planetary Science,690
2,Nuclear and High Energy Physics,336
3,General Physics and Astronomy,247
4,Electrical and Electronic Engineering,144


### Requête 5: Quel est le nombre de papiers publiés par publishers?

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

### Requête 6: Existe-t-il des liens de référence entre certains papiers du dataset?

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

### Requête 7

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

### Requête 8

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

### Requête 9

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

### Requête 10

In [ ]:
query_string = '''

'''
conn.query(query_string, db="paperswithcode")

## Fermeture de la Connexion

In [ ]:
conn.close()